## 音声認識モデル Wav2Vec と whisper を SageMaker 上でデプロイして試してみる



本チュートリアルでは、音声認識モデルである wav2vec2 や whisper を Studio Lab 経由で、AWS 環境上にデプロイする流れを体験してみます。  
Amazon SageMaker 上で Hugging Face Inference DLC を使って MetaAI の [wav2vec2](https://arxiv.org/abs/2006.11477) や OpenAI の [whisper](https://cdn.openai.com/papers/whisper.pdf) モデルを手軽に利用することが可能です。  

このサンプルでは、
- Studio Lab 上で Amazon SageMaker の機能を利用する設定を行う
- Studio Lab 経由で Amazon SageMaker 上に `transformers` のモデルをデプロイする
- デプロイしたモデルに対して音声ファイルを投げて推論させてみる

の３つを各モデルについて行っていきます。  
なお、このサンプルは [Automatic Speech Recogntion with Hugging Face's Transformers & Amazon SageMaker](https://github.com/huggingface/notebooks/blob/main/sagemaker/20_automatic_speech_recognition_inference/sagemaker-notebook.ipynb) と[Sentence Embeddings with Hugging Face Transformers, Sentence Transformers and Amazon SageMaker - Custom Inference for creating document embeddings with Hugging Face's Transformers](https://github.com/huggingface/notebooks/blob/main/sagemaker/17_custom_inference_script/sagemaker-notebook.ipynb)をベースに作成しています。  

## 環境のセットアップ

### モジュールのインストール

インストールが完了したら、カーネルを再起動することを忘れないでください。  

In [ ]:
!pip install -U boto3
!pip install -U sagemaker
!pip install -U transformers

### AWS 環境のセットアップ


下記のハンズオン資料の「2-2. AWS へ接続するための環境構築」「2-3. SageMaker Training Instance が利用する IAM ロールを作成する」で紹介されている手順にそって作業を進めてください。  

- https://github.com/aws-samples/aws-ml-enablement-workshop/blob/main/notebooks/scenario_churn/customer_churn_sagemaker.ipynb 


作業が完了して作成された Role の ARN の値を以下のセルで置き換えてください。

In [ ]:
role = "arn:aws:iam::000000000000:role/StudioLabWhisperExecutionRole"  # TODO: コピペした値で置き換える

## Wav2Vec のデプロイ

まずは、 SageMaker SDK の HuggingFace 拡張を使って簡単にモデルをデプロイしてみましょう。  
以下のページで公開されている「wav2vec2-large-960h」と呼ばれているモデルを使っていきます。  

- https://huggingface.co/facebook/wav2vec2-large-960h

この HuggingFace のページ上で SageMaker でモデルをデプロイするためのコードを手軽に生成できます。  

まずは、ページにある「Deploy」ボタンをクリックします。  

![](./imgs/101_wav2vec_deploy_button.png)

いくつかデプロイの選択肢が出てくるので今回は「Amazon SageMaker」を選択します。  

![](./imgs/102_wav2vec_deploy_select_sagemaker.png)

Task を「Automatic Speech Recognition」、Configuration を「AWS」に設定すると deploy 用のコードが生成されます。  

![](./imgs/103_wav2vec_deploy_generate_code.png)



コピーしたコードはデプロイのコードと推論のコードが含まれています。  
このままでは動かないため、audio_serializer の追加や IAM Role 部分のコメントアウトなどをする必要があります。  

ここで使用している `DataSerializer` は推論リクエストの際に、音声ファイルをリクエストする手順を簡易化してくれます。  


注意！！！  
**以下のコードを実行すると、SageMaker 上でエンドポイントがデプロイされます。  
エンドポイントは時間課金されるため、エンドポイントの消し忘れに注意してください。**  
意図せぬ課金を避けるためにも、最後の後片付けのステップは必ず実施してください。  

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.serializers import DataSerializer
import sagemaker

#### <変更箇所> 下の行はコメントアウトし、最初に設定した値を使う。 ####
# role = sagemaker.get_execution_role()
#### 
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'facebook/wav2vec2-base-960h',
	'HF_TASK':'automatic-speech-recognition'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.17.0',
	pytorch_version='1.10.2',
	py_version='py38',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
#### <変更箇所> 音声データをリクエストできるよう Audio Seriealizer を追加する ####
audio_serializer = DataSerializer(content_type='audio/x-audio')  # x-audio にしておくことで複数の音声フォーマットに対応
####
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge', # ec2 instance type
    #### <変更箇所> Audio Serializer を追加 ####
    serializer=audio_serializer
    ####
)

#### <変更箇所> 推論部分は別途実装する必要があるのでコメントアウト
# predictor.predict({
# 	'inputs': "sample1.flac"
# })
####



### デプロイしたエンドポイントに対して音声データを送ってみる

まずは、公開されている音声データを使ってどんな結果が返ってくるかみてみましょう。

huggingface.io 上で公開されている `libirispeech` と呼ばれる音声データセットの中からファイルを取得します。

In [ ]:
!wget https://cdn-media.huggingface.co/speech_samples/sample1.flac

In [ ]:
audio_path = "sample1.flac"

res = predictor.predict(data=audio_path)
print(res)

また、音声データをバイナリ形式で送信する方法もあります。

In [ ]:
audio_path = "sample1.flac"

with open(audio_path, "rb") as data_file:
    audio_data = data_file.read()
    res = predictor.predict(data=audio_data)
    print(res)

### 後片付け

先ほど作成したモデルとデプロイしたエンドポイントを最後に削除します。

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

## whisper のデプロイ

ここからは、wav2vec ではなく OpenAI から出された音声書き起こしモデルである whisper を使う方法を紹介していきます。  
現状、HuggingFace DLC は [v4.17](https://github.com/huggingface/transformers/releases/tag/v4.17.0) 対応となっているのですが whisper は [v4.23.1](https://github.com/huggingface/transformers/releases/tag/v4.23.1) 以降でないと使うことができません。  

このためには `requirements.txt` で `transformers==4.23.1` を追加する必要があります。その手順を見ていきましょう。  
[こちらの Notebook](https://github.com/huggingface/notebooks/blob/main/sagemaker/17_custom_inference_script/sagemaker-notebook.ipynb) で記載されている手順をベースに考えていきます。

まずは、 `requirements.txt` を格納するためのフォルダを作成します。

In [ ]:
!mkdir code

In [ ]:
%%writefile code/requirements.txt
transformers==4.23.1

`requirements.txt` を上書きした推論コードをアップロードします。

In [ ]:
import sagemaker
sess = sagemaker.Session()
default_bucket = sess.default_bucket()
# default_bucket = "<BUCKET_NAME>"  # もし自前で作成したバケットを使う場合はこちらのコメントを解除して値を指定してください
print(default_bucket)

In [ ]:
repository = "openai/whisper-base"
model_id=repository.split("/")[-1]
s3_location=f"s3://{default_bucket}/custom_inference/{model_id}/model.tar.gz"

モデルを `git clone` で hf.co/models からダウンロードします。

In [ ]:
!git lfs install
!git clone https://huggingface.co/$repository

先ほど作成した `requirements.txt` をコピーします。

In [ ]:
!pwd

In [ ]:
!cp -r code/ $model_id/code/

`requirements.txt` やモデルアーティファクトを含める形で `model.tar.gz` を作成します。

In [ ]:
%cd $model_id
!tar zcvf model.tar.gz *

`model.tar.gz` を s3 にアップロードします。

In [ ]:
!aws s3 cp model.tar.gz $s3_location

### モデルのデプロイ

先ほどアップロードしたモデルを使って deploy をしてみます。  
wav2vec での例との差分は、model_data の値を追加で指定しているところです。

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.serializers import DataSerializer
import sagemaker


#### <変更箇所> hub の値は使わない ####
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'openai/whisper-base',
	'HF_TASK':'automatic-speech-recognition'
}
####

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    #### <変更箇所> model_data の値として s3_location を使う ####
    model_data = s3_location,
    ####
	transformers_version='4.17.0',
	pytorch_version='1.10.2',
	py_version='py38',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
audio_serializer = DataSerializer(content_type='audio/x-audio')
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge', # ec2 instance type
    serializer=audio_serializer
)

### 推論リクエストを投げてみる
元のフォルダに移動します

In [ ]:
%cd ..

(wav2vec のサンプルを実行していなければ) 下記のコメントアウトを外して音声ファイルのダウンロードを行います

In [ ]:
# !wget https://cdn-media.huggingface.co/speech_samples/sample1.flac

書き起こしがうまく動いていそうか確認します。

In [ ]:
audio_path = "sample1.flac"

res = predictor.predict(data=audio_path)
print(res)

### 後片付け

先ほど作成したモデルとデプロイしたエンドポイントを最後に削除します。

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

In [ ]:
!aws s3 rm $s3_location